# 国立アイヌ民族博物館アイヌ語アーカイブ

In [1]:
from pathlib import Path
DIGITAL_ARCHIVE_DIR = Path('../input/archive/')

In [5]:
import regex as re

from bs4 import BeautifulSoup


columns = ['lemma', 'def', 'pron', 'tag']
dictionary = set()

from tqdm.notebook import tqdm

for file in tqdm(list(DIGITAL_ARCHIVE_DIR.glob('*.html'))):
    print(file)
    with open(file, 'r') as f:
        file_tag = file.stem
        
        # remove unnecessary spaces
        html = f.read().replace('\n', '')
        html = re.sub(r'\s+', ' ', html)

        soup = BeautifulSoup(html, 'html.parser')
        dl = soup.find('dl')
        if not dl:
            raise Exception(f'No dl element found in {file}')
        for dt, dd in tqdm(list(zip(dl.find_all('dt'), dl.find_all('dd')))):
            # Extract the key and sound URL from the dt element
            if sup := dt.find('sup'):
                sup.extract()
            key = dt.text.strip()
            
            sound_a = dt.find('a', class_='sound')
            sound_url = sound_a['data-sound-url'] if sound_a else None
            
            # Extract the value from the dd element
            value = dd.text.strip()

            dictionary.add((key, value, sound_url, file_tag))

import pandas as pd

pd.DataFrame(dictionary, columns=columns).dropna(subset=['lemma']).reset_index(drop=True).sort_values('lemma').reset_index(drop=True).drop_duplicates().reset_index(drop=True).to_csv(DIGITAL_ARCHIVE_DIR / 'dictionary.csv', index=False)


  0%|          | 0/6 [00:00<?, ?it/s]

data\archive\-.html


  0%|          | 0/19049 [00:00<?, ?it/s]

data\archive\a.html


  0%|          | 0/20273 [00:00<?, ?it/s]

...

  0%|          | 0/16815 [00:00<?, ?it/s]

data\archive\o.html


  0%|          | 0/15886 [00:00<?, ?it/s]

data\archive\u.html


  0%|          | 0/14903 [00:00<?, ?it/s]

In [9]:
import pandas as pd
import regex as re
df = pd.read_csv(DIGITAL_ARCHIVE_DIR / 'dictionary.csv').dropna(subset=['lemma'])
# df[df['def'].apply(lambda x: re.search(r'^[\p{Katakana}\-]', x) is None)].to_csv('./result.csv')
df[df['def'].apply(lambda x: re.search(r'（出典：.*?、方言：.*?）$', x) is None)].to_csv('./result.csv')

KeyboardInterrupt: 

In [4]:
# count all words in the dictionary in pos column   

KeyError: 'pos'

### Clean up the dictionary

In [2]:
import pandas as pd
from pathlib import Path
df_archive = (
    pd.read_csv(DIGITAL_ARCHIVE_DIR/ "dictionary.csv")
    .dropna(subset=["lemma"])
    .drop_duplicates(subset=["lemma", "def"])
    .reset_index(drop=True)
)

HALF_WIDTH_KANA_2_SMALL_KANA = {
    "ｸ": "ㇰ",
    "ｼ": "ㇱ",
    "ｽ": "ㇲ",
    "ﾄ": "ㇳ",
    "ﾇ": "ㇴ",
    "ﾊ": "ㇵ",
    "ﾋ": "ㇶ",
    "ﾌﾟ": "ㇷ゚",
    "ﾌ": "ㇷ",
    "ﾍ": "ㇸ",
    "ﾎ": "ㇹ",
    "ﾑ": "ㇺ",
    "ﾗ": "ㇻ",
    "ﾘ": "ㇼ",
    "ﾙ": "ㇽ",
    "ﾚ": "ㇾ",
    "ﾛ": "ㇿ",
}


# Replace half-width katakana with small katakana
df_archive['def'] = df_archive['def'].apply(lambda x: re.sub(rf'({"|".join(HALF_WIDTH_KANA_2_SMALL_KANA.keys())})', lambda m: HALF_WIDTH_KANA_2_SMALL_KANA[m.group(1)], x))


FileNotFoundError: [Errno 2] No such file or directory: 'data/archive/dictionary.csv'

In [12]:
df_archive[["lemma", "def"]].to_csv(
    DIGITAL_ARCHIVE_DIR / "cleaned-dictionary.tsv", sep="\t", index=False
)

In [13]:
import pandas as pd
from pathlib import Path
cleaned_dictionary = pd.read_csv(DIGITAL_ARCHIVE_DIR/ 'cleaned-dictionary.tsv', sep='\t')

print(len(cleaned_dictionary))

pd.set_option('display.max_colwidth',1000)
cleaned_dictionary.head()

26904


,lemma,def
0,(a-)ekonramu-sitne,"エコンラムシッネ §245．きぜつする（気絶する）；気が遠くなる；気を失う；人事不省になる(13)うつらうつらする[気を失ってから我に返る直前の半意識の状態にある](a-)ekonramu-sitne〔e-kón-ra-mu-šit-ne エこンラム･シッネ〕[a-(我)、e(それについて)、kon(<kor 持っている)、ramu(その心･心臓〔が〕)、sit(糸などのもつれた玉)、ne(のようになる)]｟ホロベツ,サル―ユ研Ⅱ, pp.466〜467｠ （出典：知里人間編I、方言：）"
1,(a-)ekonramu-tanak-tanak,エコンラムタナㇰタナㇰ §245．きぜつする（気絶する）；気が遠くなる；気を失う；人事不省になる(12)うつらうつらする[気を失ってから我に返ろうとする前の半意識の状態、ゆめうつつの状態にある](a-)ekonramu-tanak-tanak〔エこンラム･たナㇰタナㇰ〕[a-(我)、e(それについて)、kon(<kor もっている、自分の)、ramu(その心〔が〕]、tanak-tanak(うつらうつらする)>｟サル―ユ研Ⅱ、 p.599｠ （出典：知里人間編I、方言：）
2,-a,ア 【接尾】[名詞の所属形形成](所属語尾とも呼ぶ。 母音 a ア + 子音に終る語幹のうちのあるものにつく。 この母音がついて所属形の｢短い形｣ができる。 さらにその後に ha ハ がついて(つまり語幹に -aha アハ がついて)所属形の｢長い形｣がつくられる。 本辞典では長短二つの形を合わせて…a(ha) のように表記してある｡) sar サㇻ [概]尾；sara(ha) サラ(ハ) [所]…の尾。 （出典：田村、方言：沙流）
3,-aha,アハ 【接尾】[名詞の所属形形成](｢所属語尾｣とも呼ぶ。 所属形の｢長い形｣をつくる｡) ☞-a ア （出典：田村、方言：沙流）
4,-ar,アㇻ 【接尾】[不定使役形接尾辞](＝-yar ヤㇻ)(子音の後で y が落ちた形。 母音の後では -yar ヤㇻ の形が使われる｡) kar カㇻ …をつくる；karar カラㇻ …を人につくってもらう。 （出典：田村、方言：沙流）


In [15]:
# Find all 田村's in the dictionary (ending with 田村)

tamura_entries = cleaned_dictionary.loc[cleaned_dictionary['def'].str.contains('（出典：田村、方言：沙流）')]

# Remove the citation from the definition
tamura_entries.loc[:, 'def'] = tamura_entries['def'].apply(lambda x: re.sub(r'（出典：田村、方言：沙流）', '', x))
tamura_entries

C:\Users\mk\AppData\Local\Temp\ipykernel_44392\1546722303.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tamura_entries.loc[:, 'def'] = tamura_entries['def'].apply(lambda x: re.sub(r'（出典：田村、方言：沙流）', '', x))


In [18]:
tamura_entries['translit'] = tamura_entries['def'].apply(lambda x: re.match(r'(.*?)(【.*)', x).group(1) if re.match(r'(.*?)(【.*)', x) else re.match(r'(\p{scx=Katakana}+)', x).group(1))
tamura_entries['def'] = tamura_entries['def'].apply(lambda x: re.match(r'(.*?)(【.*)', x).group(2) if re.match(r'(.*?)(【.*)', x) else re.match(r'(\p{scx=Katakana}+)', x).group(1))
tamura_entries = tamura_entries[['lemma', 'translit', 'def']]

C:\Users\mk\AppData\Local\Temp\ipykernel_44392\2980672500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tamura_entries['translit'] = tamura_entries['def'].apply(lambda x: re.match(r'(.*?)(【.*)', x).group(1) if re.match(r'(.*?)(【.*)', x) else re.match(r'(\p{scx=Katakana}+)', x).group(1))
C:\Users\mk\AppData\Local\Temp\ipykernel_44392\2980672500.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tamura_entries['def'] = tamura_entries['def'].apply(lambda x: re.match(r'(.*?)(【.*)', x).group(2) if re.matc

In [21]:
tamura_entries.to_csv(DIGITAL_ARCHIVE_DIR / "tamura-entries.tsv", sep="\t", index=False)

In [3]:
tamura_entries = pd.read_csv(DIGITAL_ARCHIVE_DIR / "tamura-entries.tsv", sep="\t")
tamura_entries

,lemma,translit,def
0,-a,ア,【接尾】[名詞の所属形形成](所属語尾とも呼ぶ。 母音 a ア + 子音に終る語幹のうちのあ...
1,-aha,アハ,【接尾】[名詞の所属形形成](｢所属語尾｣とも呼ぶ。 所属形の｢長い形｣をつくる｡) ☞-a ア
2,-ar,アㇻ,【接尾】[不定使役形接尾辞](＝-yar ヤㇻ)(子音の後で y が落ちた形。 母音の後では...
3,-asnu,アㇱヌ,【接尾】(語根またはそれから派生した語基に接尾して､ 性質を表す自動詞(形容詞)をつくる｡)...
4,-atki,アッキ,【接尾】(自動詞をつくる。 限られた決まった語にのみ現われる｡)tusus トゥスㇱ (擬態...
...,...,...,...
9922,útek,ウテㇰ,【他動】(人)を使いにやる､ (人)を(使い走りの仕事に)使う。 póho útek haw...
9923,útur,ウトゥㇽ,【位名】[概](所は útur(u)ke(he) ウトゥル/ウトゥㇽ/ウトゥルケ(ヘ)/ウト...
9924,útur(u)ke(he),ウトゥㇽケ(ヘ)/ウトゥルケ(ヘ),【位名】[所](概は útur ウトゥㇽ)[utur-u-ke …の下座・(所属語尾)・(よ...
9925,úturke,ウトゥㇽケ,【位名】☞útur(u)ke(he) ウトゥㇽケ(ヘ)/ウトゥルケ(ヘ)
